In [1]:
import preprocess

In [2]:
tst = [2023, 8, 28, 'AM']
ted = [2023, 9, 6, 'AM']
region = ('CONUS', [-126.401171875,24.071240929282325,
                    -61.36210937500001,49.40003415463647])

In [3]:
import FireTime

list_of_ts = list(FireTime.t_generator(tst, ted))
list_of_ts

[[2023, 8, 28, 'AM'],
 [2023, 8, 28, 'PM'],
 [2023, 8, 29, 'AM'],
 [2023, 8, 29, 'PM'],
 [2023, 8, 30, 'AM'],
 [2023, 8, 30, 'PM'],
 [2023, 8, 31, 'AM'],
 [2023, 8, 31, 'PM'],
 [2023, 9, 1, 'AM'],
 [2023, 9, 1, 'PM'],
 [2023, 9, 2, 'AM'],
 [2023, 9, 2, 'PM'],
 [2023, 9, 3, 'AM'],
 [2023, 9, 3, 'PM'],
 [2023, 9, 4, 'AM'],
 [2023, 9, 4, 'PM'],
 [2023, 9, 5, 'AM'],
 [2023, 9, 5, 'PM'],
 [2023, 9, 6, 'AM']]

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [5]:
preprocess.preprocess_landcover()

2024-02-07 16:31:53,807 - FireLog - INFO - func:preprocess_landcover took: 6.36 sec


## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [4]:
preprocess.preprocess_region(region)

2024-02-07 16:39:55,354 - FireLog - INFO - func:preprocess_region took: 1.33 min


## Once per input file

Next process each NRT file into half day files. First we'll get all the times that are of interest. This could also be done by inspecting all looking at all the files that exist and seeing which have not been preprocessed yet.

In [5]:
%%time
for sat in ["SNPP", "NOAA20"]:
    for t in list_of_ts[::2]:
        preprocess.preprocess_NRT_file(t, sat)

2024-02-07 16:39:57,936 - FireLog - INFO - preprocessing NRT file for 2023-8-28, SNPP
2024-02-07 16:39:58,244 - FireLog - INFO - func:preprocess_NRT_file took: 307.72 ms
2024-02-07 16:39:58,245 - FireLog - INFO - preprocessing NRT file for 2023-8-29, SNPP
2024-02-07 16:39:58,519 - FireLog - INFO - func:preprocess_NRT_file took: 274.09 ms
2024-02-07 16:39:58,519 - FireLog - INFO - preprocessing NRT file for 2023-8-30, SNPP
2024-02-07 16:39:58,776 - FireLog - INFO - func:preprocess_NRT_file took: 256.48 ms
2024-02-07 16:39:58,776 - FireLog - INFO - preprocessing NRT file for 2023-8-31, SNPP
2024-02-07 16:39:59,019 - FireLog - INFO - func:preprocess_NRT_file took: 243.17 ms
2024-02-07 16:39:59,020 - FireLog - INFO - preprocessing NRT file for 2023-9-1, SNPP
2024-02-07 16:39:59,339 - FireLog - INFO - func:preprocess_NRT_file took: 318.74 ms
2024-02-07 16:39:59,339 - FireLog - INFO - preprocessing NRT file for 2023-9-2, SNPP
2024-02-07 16:39:59,720 - FireLog - INFO - func:preprocess_NRT_fil

CPU times: user 5.93 s, sys: 124 ms, total: 6.05 s
Wall time: 6.05 s


## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

Note: for the purpose of timing I am running the full month, but each of these steps could run in a separate proccess.

In [4]:
region = preprocess.read_region(region)

2024-02-07 16:43:11,041 - FireLog - INFO - func:read_region took: 182.86 ms


In [5]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="VIIRS", region=region)

2024-02-07 16:43:11,314 - FireLog - INFO - filtering and clustering 2023-8-28 AM, VIIRS, CONUS
2024-02-07 16:43:11,334 - FireLog - INFO - func:read_preprocessed took: 17.89 ms
2024-02-07 16:43:11,348 - FireLog - INFO - func:read_preprocessed took: 13.20 ms
2024-02-07 16:44:21,109 - FireLog - INFO - func:do_clustering took: 518.07 ms
2024-02-07 16:44:21,139 - FireLog - INFO - func:preprocess_region_t took: 1.16 min
2024-02-07 16:44:21,140 - FireLog - INFO - filtering and clustering 2023-8-28 PM, VIIRS, CONUS
2024-02-07 16:44:21,177 - FireLog - INFO - func:read_preprocessed took: 36.71 ms
2024-02-07 16:44:21,226 - FireLog - INFO - func:read_preprocessed took: 48.48 ms
2024-02-07 16:45:07,423 - FireLog - INFO - func:do_clustering took: 17.78 ms
2024-02-07 16:45:07,438 - FireLog - INFO - func:preprocess_region_t took: 46.30 sec
2024-02-07 16:45:07,438 - FireLog - INFO - filtering and clustering 2023-8-29 AM, VIIRS, CONUS
2024-02-07 16:45:07,457 - FireLog - INFO - func:read_preprocessed too

CPU times: user 12min 41s, sys: 299 ms, total: 12min 42s
Wall time: 12min 45s
